In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
HYGVol = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Adj HYG")
LQDyas = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="LQD")
HYGyas = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HYG")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")
SPY_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPY")

portfolio = portfolio.set_index("Date")
HYGVol = HYGVol.set_index("Date").sort_index()
HYGVol = HYGVol.loc[:,"Volume"]
LQDyas = LQDyas.set_index("Date").sort_index()
HYGyas = HYGyas.set_index("Date").sort_index()
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()
SPY_Idx = SPY_Idx.set_index("Date").sort_index()

portfolio.loc[:,"HYGCumDiv"]=portfolio.loc[:,"HYG Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueHYG"]=portfolio.loc[:,"HYG Position"]+portfolio.loc[:,"HYGCumDiv"]
portfolio = portfolio.sort_index()

HYGCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueHYG"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
HYGCorrData.loc[:,"TotalReturnsHYG"] = HYGCorrData["TotalValueHYG"].pct_change()

HYGCorrData = HYGCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS"})
HYGCorrData.loc[:,"HY_Index_OAS_Diff"]=HYGCorrData["HY_Index_OAS"].diff()

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
HYGCorrData.loc[:,"10yYieldDiff"] = HYGCorrData["10y_UST_INDEX_PX"].diff()

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_Spread"})
HYGCorrData.loc[:,"IG_Index_Z_Spread_Diff"]=(HYGCorrData["IG_INDEX_Z_Spread"].diff())

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(HYGVol.loc["2021-01-11":"2025-12-31"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"Volume":"HYG_Volume"})

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(SPY_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"SPY_Idx_LAST_PX"})
HYGCorrData.loc[:,"ReturnSPY_Idx"] = HYGCorrData["SPY_Idx_LAST_PX"].pct_change()

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(HYGyas.loc["2021-01-11":"2025-12-31","YAS_MOD_DUR"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"YAS_MOD_DUR":"HYG_MOD_DUR_YAS"})

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

HYGCorrData = HYGCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
HYGCorrData = HYGCorrData.rename(columns={"PX_LAST":"Vix Idx"})

HYGCorrData=HYGCorrData.dropna()
HYGCorrData = HYGCorrData.sort_index(ascending=False)


In [12]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(HYGCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPY_Idx","MOVE Idx","Vix Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPY_Idx","MOVE Idx","Vix Idx"]))

X_pca = pcav1.transform(X_Std)

HYGCorrData.loc[:,"PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"PC2"]=X_pca[:,1]
HYGCorrData.loc[:,"PC3"]=X_pca[:,2]
HYGCorrData.loc[:,"PC4"]=X_pca[:,3]
HYGCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(HYGCorrData[["PC1","PC2","PC3","PC4"]])
Y = HYGCorrData["TotalReturnsHYG"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.39707456 0.20850729 0.1692098  0.11470395 0.07174926 0.03875513]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_Z_Spread_Diff  ReturnSPY_Idx  \
0           0.581172     -0.163460                0.566444      -0.486361   
1          -0.175898      0.465330               -0.121508      -0.053439   
2          -0.002617      0.757975                0.009043      -0.430689   
3          -0.083715     -0.163153               -0.061582       0.135200   
4           0.182591      0.373773                0.554277       0.719507   
5           0.768729      0.126087               -0.594373       0.197826   

   MOVE Idx   Vix Idx  
0  0.113516  0.255398  
1  0.622952  0.588930  
2 -0.469168 -0.140622  
3 -0.614136  0.753086  
4 -0.041945 -0.016779  
5 -0.001792  0.027190  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.726
Method:                 Least Squares   F-statistic:                     822.5
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:21:46   Log-Likelihood:                 5785.6
No. Observations:                1240   AIC:                        -1.156e+04
Df Residuals:                    1235   BIC:                        -1.154e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001   6.48e-05      2.026      0.043    4.14e-06       0.000
PC1           -0.0016    4.2e-05    -37.569      0.000      -0.002      -0.001
PC2           -0.0007   5.79e-05    -11.479      0.000      -0.001      -0.001
PC3           -0.0026   6.43e-05    -40.192      0.000      -0.003      -0.002
PC4            0.0009   7.81e-05     11.462      0.000       0.001       0.001
==============================================================================
Omnibus:                       91.349   Durbin-Watson:                   2.077
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              309.129
Skew:                          -0.302   Prob(JB):                     7.47e-68
Kurtosis:                       5.370   Cond. No.                         1.86
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(HYGCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPY_Idx","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPY_Idx","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

HYGCorrData.loc[:,"PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"PC2"]=X_pca[:,1]
HYGCorrData.loc[:,"PC3"]=X_pca[:,2]
HYGCorrData.loc[:,"PC4"]=X_pca[:,3]
HYGCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(HYGCorrData[["PC1","PC2","PC3","PC4"]])
Y = HYGCorrData["TotalReturnsHYG"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.45795968 0.21291247 0.19641167 0.08612006 0.04659612]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_Z_Spread_Diff  ReturnSPY_Idx  \
0           0.606673     -0.193439                0.585946      -0.496454   
1          -0.064401      0.835424               -0.019302      -0.371688   
2          -0.013997     -0.331673               -0.000881       0.237555   
3           0.180063      0.371435                0.553468       0.721686   
4           0.771480      0.129144               -0.591580       0.195241   

   MOVE Idx  
0  0.068776  
1  0.399244  
2  0.912888  
3 -0.049554  
4  0.007373  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     833.2
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:13:58   Log-Likelihood:                 5791.4
No. Observations:                1240   AIC:                        -1.157e+04
Df Residuals:                    1235   BIC:                        -1.155e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001   6.45e-05      2.035      0.042    4.74e-06       0.000
PC1           -0.0016   4.26e-05    -37.549      0.000      -0.002      -0.002
PC2           -0.0024   6.25e-05    -38.532      0.000      -0.003      -0.002
PC3            0.0013   6.51e-05     20.321      0.000       0.001       0.001
PC4            0.0005   9.83e-05      5.021      0.000       0.000       0.001
==============================================================================
Omnibus:                       74.886   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              299.145
Skew:                          -0.061   Prob(JB):                     1.10e-65
Kurtosis:                       5.403   Cond. No.                         2.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<u> I Like this one more: </u>

In [8]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(HYGCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPY_Idx","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPY_Idx","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

HYGCorrData.loc[:,"PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"PC2"]=X_pca[:,1]
HYGCorrData.loc[:,"PC3"]=X_pca[:,2]
HYGCorrData.loc[:,"PC4"]=X_pca[:,3]
HYGCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(HYGCorrData[["PC1","PC2","PC3"]])
Y = HYGCorrData["TotalReturnsHYG"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.45795968 0.21291247 0.19641167 0.08612006 0.04659612]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_Z_Spread_Diff  ReturnSPY_Idx  \
0           0.606673     -0.193439                0.585946      -0.496454   
1          -0.064401      0.835424               -0.019302      -0.371688   
2          -0.013997     -0.331673               -0.000881       0.237555   
3           0.180063      0.371435                0.553468       0.721686   
4           0.771480      0.129144               -0.591580       0.195241   

   MOVE Idx  
0  0.068776  
1  0.399244  
2  0.912888  
3 -0.049554  
4  0.007373  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.723
Method:                 Least Squares   F-statistic:                     1081.
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:15:20   Log-Likelihood:                 5778.9
No. Observations:                1240   AIC:                        -1.155e+04
Df Residuals:                    1236   BIC:                        -1.153e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001   6.51e-05      2.016      0.044    3.51e-06       0.000
PC1           -0.0016    4.3e-05    -37.187      0.000      -0.002      -0.002
PC2           -0.0024   6.31e-05    -38.160      0.000      -0.003      -0.002
PC3            0.0013   6.57e-05     20.125      0.000       0.001       0.001
==============================================================================
Omnibus:                       78.840   Durbin-Watson:                   2.065
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              311.973
Skew:                          -0.128   Prob(JB):                     1.80e-68
Kurtosis:                       5.444   Cond. No.                         1.53
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(HYGCorrData[["ReturnSPY_Idx","Vix Idx","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["ReturnSPY_Idx","Vix Idx","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]))

X_pca = pcav1.transform(X_Std)

HYGCorrData.loc[:,"PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"PC2"]=X_pca[:,1]
HYGCorrData.loc[:,"PC3"]=X_pca[:,2]
HYGCorrData.loc[:,"PC4"]=X_pca[:,3]
HYGCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(HYGCorrData[["PC1","PC2","PC3","PC4"]])
Y = HYGCorrData["TotalReturnsHYG"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.47299414 0.224027   0.17022178 0.08625049 0.04650659]
   ReturnSPY_Idx   Vix Idx  10yYieldDiff  IG_Index_Z_Spread_Diff  \
0      -0.491459  0.237983     -0.169757                0.571853   
1      -0.297426  0.488585      0.804992               -0.081057   
2       0.339935  0.838115     -0.407663               -0.070596   
3       0.717898 -0.038825      0.375626                0.555007   
4       0.197665  0.026530      0.126042               -0.594473   

   HY_Index_OAS_Diff  
0           0.588217  
1          -0.135055  
2          -0.104087  
3           0.184354  
4           0.768726  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.732
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     843.5
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:16:06   Log-Likelihood:                 5797.0
No. Observations:                1240   AIC:                        -1.158e+04
Df Residuals:                    1235   BIC:                        -1.156e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001   6.42e-05      2.045      0.041    5.31e-06       0.000
PC1           -0.0016   4.18e-05    -38.138      0.000      -0.002      -0.002
PC2           -0.0020   6.07e-05    -33.345      0.000      -0.002      -0.002
PC3            0.0020   6.96e-05     28.019      0.000       0.002       0.002
PC4            0.0005   9.78e-05      4.744      0.000       0.000       0.001
==============================================================================
Omnibus:                       79.888   Durbin-Watson:                   2.042
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              283.945
Skew:                          -0.208   Prob(JB):                     2.20e-62
Kurtosis:                       5.307   Cond. No.                         2.34
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(HYGCorrData[["ReturnSPY_Idx","Vix Idx","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["ReturnSPY_Idx","Vix Idx","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]))

X_pca = pcav1.transform(X_Std)

HYGCorrData.loc[:,"PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"PC2"]=X_pca[:,1]
HYGCorrData.loc[:,"PC3"]=X_pca[:,2]
HYGCorrData.loc[:,"PC4"]=X_pca[:,3]
HYGCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(HYGCorrData[["PC1","PC2","PC3"]])
Y = HYGCorrData["TotalReturnsHYG"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.47299414 0.224027   0.17022178 0.08625049 0.04650659]
   ReturnSPY_Idx   Vix Idx  10yYieldDiff  IG_Index_Z_Spread_Diff  \
0      -0.491459  0.237983     -0.169757                0.571853   
1      -0.297426  0.488585      0.804992               -0.081057   
2       0.339935  0.838115     -0.407663               -0.070596   
3       0.717898 -0.038825      0.375626                0.555007   
4       0.197665  0.026530      0.126042               -0.594473   

   HY_Index_OAS_Diff  
0           0.588217  
1          -0.135055  
2          -0.104087  
3           0.184354  
4           0.768726  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.726
Method:                 Least Squares   F-statistic:                     1098.
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:16:27   Log-Likelihood:                 5785.8
No. Observations:                1240   AIC:                        -1.156e+04
Df Residuals:                    1236   BIC:                        -1.154e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001   6.48e-05      2.027      0.043    4.22e-06       0.000
PC1           -0.0016   4.21e-05    -37.811      0.000      -0.002      -0.002
PC2           -0.0020   6.12e-05    -33.059      0.000      -0.002      -0.002
PC3            0.0020   7.02e-05     27.778      0.000       0.002       0.002
==============================================================================
Omnibus:                       87.692   Durbin-Watson:                   2.074
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              302.896
Skew:                          -0.273   Prob(JB):                     1.69e-66
Kurtosis:                       5.359   Cond. No.                         1.67
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(HYGCorrData[["ReturnSPY_Idx","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["ReturnSPY_Idx","10yYieldDiff","IG_Index_Z_Spread_Diff","HY_Index_OAS_Diff"]))

X_pca = pcav1.transform(X_Std)

HYGCorrData.loc[:,"PC1"]=X_pca[:,0]
HYGCorrData.loc[:,"PC2"]=X_pca[:,1]
HYGCorrData.loc[:,"PC3"]=X_pca[:,2]
HYGCorrData.loc[:,"PC4"]=X_pca[:,3]

X = sm.add_constant(HYGCorrData[["PC1","PC2","PC3"]])
Y = HYGCorrData["TotalReturnsHYG"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.57091853 0.26282895 0.10799706 0.05825545]
   ReturnSPY_Idx  10yYieldDiff  IG_Index_Z_Spread_Diff  HY_Index_OAS_Diff
0      -0.497613     -0.195779                0.586678           0.608162
1      -0.434869      0.898860               -0.014838          -0.052146
2       0.724544      0.369994                0.553623           0.177882
3       0.195719      0.129715               -0.590839           0.771866


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.728
Method:                 Least Squares   F-statistic:                     1108.
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:20:19   Log-Likelihood:                 5790.0
No. Observations:                1240   AIC:                        -1.157e+04
Df Residuals:                    1236   BIC:                        -1.155e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001   6.45e-05      2.034      0.042    4.64e-06       0.000
PC1           -0.0016   4.27e-05    -37.540      0.000      -0.002      -0.002
PC2           -0.0027    6.3e-05    -43.446      0.000      -0.003      -0.003
PC3            0.0005   9.82e-05      5.267      0.000       0.000       0.001
==============================================================================
Omnibus:                       75.309   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              307.272
Skew:                           0.018   Prob(JB):                     1.89e-67
Kurtosis:                       5.438   Cond. No.                         2.30
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""